In [9]:
import requests
from bs4 import BeautifulSoup as bs
import collections
import time

In [10]:
HOST = 'https://chita.hh.ru/'

URL = 'https://chita.hh.ru/search/vacancy?st=searchVacancy&text=data+engineer&area=113&salary=&currency_code=RUR&' \
      'experience=doesNotMatter&schedule=remote&order_by=publication_time&search_period=3&items_on_page=20&' \
      'no_magic=true&L_save_area=true&page='

HEADERS = {
    'Host': 'chita.hh.ru',
    'User-agent': 'Mozilla/5.0',
    'Accept': '*/*',
    'Content-type': 'application/text',
    'Connection': 'keep-alive'
}

In [13]:
def get_page_bs(url, params=''):
    page_obj = requests.get(url + '0', headers=HEADERS, params=params)
    page_txt = page_obj.text
    page_bs = bs(page_txt)
    return page_bs

In [14]:
page_bs = get_page_bs(URL)
# print(page_bs)

In [15]:
# количество страниц пагинации
def get_pages_num(page_bs):
    try:
        return int(page_bs.find_all("span", class_="pager-item-not-in-short-range")[-1].text.strip())
    except:
        # нет paginator'a, значит, есть только одна страница
        return 1

PAGES = get_pages_num(page_bs)
print(PAGES)

11


In [54]:
def get_all_pages(base_url):
    print('Загружаю первую страницу')
    page_0_bs = get_page_bs(base_url)
    yield page_0_bs
    
    n = get_pages_num(page_0_bs)
    for i in range(1, n):
        url = base_url + '&p=' + str(i)
        print('Загружаю страницу номер', i)
        page_i_bs = get_page_bs(url)
        yield(page_i_bs)

### Разбор одной страницы

In [172]:
def get_data(page_bs):
    # Соберём в виде словаря списков, каждый список - отдельная колонка
    res = collections.defaultdict(list)
    
    #
    names = ['vacancy-serp-item']
    tags = page_bs.find_all('div', class_=names[0])

    for tag in tags:
        vacancy_name = tag.find("a", class_="bloko-link")  # vacancy name
        vacancy_compensation = tag.find("div", class_="vacancy-serp-item__sidebar").text
        if vacancy_compensation is None:
            vacancy_compensation = '-'
               
        vacancy_url = vacancy_name["href"] # link vacancy
        if vacancy_url is None:
            vacancy_url = '-'
        
        vacancy_org = tag.find("a", class_="bloko-link bloko-link_secondary")  # the name of organization
        
        org_link = HOST + vacancy_org["href"][1:]
        
        vacancy_place = tag.find("span", class_="vacancy-serp-item__meta-info").text.strip()
        
        vacancy_response = HOST + tag.find("div", class_="vacancy-serp-item__controls-item vacancy-serp-item__controls-item_response").find('a')['href'][1:].strip()
        
        print(vacancy_place + ' - ', 
              vacancy_compenstion + '\n', 
              vacancy_name.text.strip() + ' - ', 
              vacancy_url.strip() + '\n', 
              vacancy_org.text.strip() + ' - ', 
              org_link + '\n',
              'Откликнуться -',
              vacancy_response + '\n'
             )
  
    return res

In [173]:
res_one_page = get_data(page_bs)

Санкт-Петербург -  
 Senior DevOps/SRE Engineer -  https://chita.hh.ru/vacancy/46172004?query=data%20engineer
 VIAcode -  https://chita.hh.ru/employer/84051
 Откликнуться - https://chita.hh.ru/applicant/vacancy_response?vacancyId=46172004&hhtmFrom=vacancy_search_result

Москва -  
 Data Analyst -  https://chita.hh.ru/vacancy/45584228?query=data%20engineer
 Launch Control -  https://chita.hh.ru/employer/5502536
 Откликнуться - https://chita.hh.ru/applicant/vacancy_response?vacancyId=45584228&hhtmFrom=vacancy_search_result

Санкт-Петербург, Бухарестская -  от 140 000 руб.
 IOS developer (Удаленная работа) -  https://chita.hh.ru/vacancy/46140704?query=data%20engineer
 ООО ИТ Лаб -  https://chita.hh.ru/employer/3809433
 Откликнуться - https://chita.hh.ru/applicant/vacancy_response?vacancyId=46140704&hhtmFrom=vacancy_search_result

Москва -  от 4 000 USD
 Product Manager - Mobile -  https://chita.hh.ru/vacancy/46103170?query=data%20engineer
 Новакид РУС -  https://chita.hh.ru/employer/85171

### Разбор всех страниц

In [50]:
def scrape1(base_url):
    res = collections.defaultdict(list)
    
    for page_i_bs in get_all_pages(base_url):
        res_i = get_data(page_i_bs)
        
        # Собираем в основной словарь
        for k in res_i.keys():
            res[k] += res_i[k]
            
        # Ждём, чтобы не попасть в чёрный список
        print(f'Готово - {}. Ждём перед загрузкой следующей страницы...')
        time.sleep(1)
    
    print('готово')
    return res

In [68]:
def scrape(url):
    res = collections.defaultdict(list)
    print(res)
    for page_num in range(PAGES):
        print(URL + str(page_num + 1))
        get_page_bs(URL + str(page_num))
        # Ждём, чтобы не попасть в чёрный список
        print(f'Готово - {page_num + 1}. Ждём перед загрузкой следующей страницы...')
        time.sleep(1)
    print(f'Готово - {page_num + 1} из {PAGES}')
    return res    

In [69]:
res = scrape(URL)

defaultdict(<class 'list'>, {})
https://chita.hh.ru/search/vacancy?st=searchVacancy&text=data+engineer&area=113&salary=&currency_code=RUR&experience=doesNotMatter&schedule=remote&order_by=publication_time&search_period=3&items_on_page=20&no_magic=true&L_save_area=true&page=1
Готово - 1. Ждём перед загрузкой следующей страницы...
https://chita.hh.ru/search/vacancy?st=searchVacancy&text=data+engineer&area=113&salary=&currency_code=RUR&experience=doesNotMatter&schedule=remote&order_by=publication_time&search_period=3&items_on_page=20&no_magic=true&L_save_area=true&page=2
Готово - 2. Ждём перед загрузкой следующей страницы...
https://chita.hh.ru/search/vacancy?st=searchVacancy&text=data+engineer&area=113&salary=&currency_code=RUR&experience=doesNotMatter&schedule=remote&order_by=publication_time&search_period=3&items_on_page=20&no_magic=true&L_save_area=true&page=3
Готово - 3. Ждём перед загрузкой следующей страницы...
https://chita.hh.ru/search/vacancy?st=searchVacancy&text=data+engineer&